# Model Signature

## Column Based Signature

In [1]:
import mlflow
from pathlib import Path
mlflow.set_tracking_uri(Path.cwd().joinpath("mlruns").as_uri())
mlflow.get_tracking_uri()

'file:///C:/Users/91950/mlflowlearn/mlruns'

In [2]:
experiment = mlflow.get_experiment_by_name("Default")
mlflow.set_experiment("Default")

<Experiment: artifact_location='file:///C:/Users/91950/mlflowlearn/mlruns/224786408056124188', creation_time=1749986358261, experiment_id='224786408056124188', last_update_time=1749986358261, lifecycle_stage='active', name='Default', tags={}>

## Defining a signature

In [3]:
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema 
from mlflow.types.schema import ColSpec

from sklearn.datasets import load_iris

In [4]:
# loading the iris dataset
iris = load_iris(as_frame=True)
iris.data

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]

In [5]:
# target
iris.target

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int64

In [6]:
# schema for the feature columns
col = ColSpec(type="double", name="sepal length (cm)", required=True)
col.to_dict()

{'type': 'double', 'name': 'sepal length (cm)', 'required': True}

In [7]:
# To create a schema for the iris dataset, we need to create a list of ColSpec objects
col_specifications = [
    ColSpec(type="double", name=feature_name, required=True) for feature_name in iris.feature_names
    ]

# schema for the model input
model_input = Schema(inputs = col_specifications)

In [11]:
model_output = Schema(inputs= [ColSpec(type="integer", name="species", required=True)])

In [9]:
# model_output.to_dict()
model_input.to_dict()

[{'type': 'double', 'name': 'sepal length (cm)', 'required': True},
 {'type': 'double', 'name': 'sepal width (cm)', 'required': True},
 {'type': 'double', 'name': 'petal length (cm)', 'required': True},
 {'type': 'double', 'name': 'petal width (cm)', 'required': True}]

In [10]:
# creating the model signature
model_signature = ModelSignature(
    inputs = model_input,
    outputs= model_output
)

In [12]:
from pprint import pprint

pprint(model_signature.to_dict(), indent=2)

{ 'inputs': '[{"type": "double", "name": "sepal length (cm)", "required": '
            'true}, {"type": "double", "name": "sepal width (cm)", "required": '
            'true}, {"type": "double", "name": "petal length (cm)", '
            '"required": true}, {"type": "double", "name": "petal width (cm)", '
            '"required": true}]',
  'outputs': '[{"type": "integer", "name": "species", "required": true}]',
  'params': None}


In [13]:
model_signature.inputs.to_dict()

[{'type': 'double', 'name': 'sepal length (cm)', 'required': True},
 {'type': 'double', 'name': 'sepal width (cm)', 'required': True},
 {'type': 'double', 'name': 'petal length (cm)', 'required': True},
 {'type': 'double', 'name': 'petal width (cm)', 'required': True}]

In [14]:
model_signature.outputs.to_dict()

[{'type': 'integer', 'name': 'species', 'required': True}]

In [15]:
model_signature.params == None

True

### Example: Login model using schema

In [16]:
from sklearn.ensemble import RandomForestClassifier 

rfc = RandomForestClassifier()
rfc.fit(iris.data, iris.target)

with mlflow.start_run(run_name="model_signature") as run:
    mlflow.sklearn.log_model(rfc, "rfc", signature=model_signature)

2025/07/05 12:56:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


In [17]:
my_model = mlflow.sklearn.load_model("runs:/"+run.info.run_id+"/rfc")

In [18]:
type(my_model).__name__

'RandomForestClassifier'

In [19]:
my_model.predict(iris.data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])